In [10]:
import os
import json
import math
from PIL import Image  # Hanya menggunakan PIL untuk manipulasi gambar sederhana

# Folder yang berisi gambar dan file JSON yang berisi label
image_train_folder = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/train'
image_test_folder = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/valid'
json_train_file = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/train/annotations_color_tain.json'
json_test_file = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/valid/annotations_color_valid.json'


In [3]:
# Fungsi untuk memuat data gambar dan label dari file JSON
def load_data_from_json_and_images(image_folder, json_file):
    images = []
    labels = []
    filenames = []  # Untuk menyimpan nama file gambar
     
    # Membaca file JSON yang berisi anotasi
    with open(json_file) as f:
        data = json.load(f)

    # Membaca gambar dan label dari JSON
    for annotation in data['annotations']:
        image_filename = data['images'][annotation['image_id']]['file_name']  # Ambil nama file gambar berdasarkan image_id
        label = annotation['color_label']  # Label yang berhubungan dengan gambar
        
        # Memuat gambar dari folder
        image_path = os.path.join(image_folder, image_filename)
        image = Image.open(image_path)  # Menggunakan PIL untuk membuka gambar
        
        # Ubah ukuran gambar jika perlu (agar ukuran konsisten)
        image_resized = image.resize((128, 128))  # Mengubah ukuran gambar menjadi 64x64
        
        # Flatten gambar menjadi array 1D
        image_flattened = list(image_resized.getdata())  # Mengambil pixel data dalam urutan (R, G, B)
        image_flattened = [pixel[0] / 255 + pixel[1] / 255 + pixel[2] / 255 for pixel in image_flattened]


        images.append(image_flattened)
        labels.append(label)
    
    return images, labels

In [12]:
# Fungsi Euclidean Distance
def euclidean_distance(vec1, vec2):
    # Menghitung jarak Euclidean antara dua vektor
    return math.sqrt(sum((a - b) ** 2 for a, b in zip(vec1, vec2)))


In [13]:
# Fungsi KNN Manual
def knn_classify(test_data, train_data, train_labels, k=3):
    distances = []
    
    # Menghitung jarak Euclidean antara data uji dan setiap data latih
    for i in range(len(train_data)):
        dist = euclidean_distance(test_data, train_data[i])
        distances.append((dist, train_labels[i]))
    
    # Urutkan berdasarkan jarak dan ambil k terdekat
    distances.sort(key=lambda x: x[0])
    k_nearest = distances[:k]
    
    # Voting label terbanyak
    label_votes = {}
    for _, label in k_nearest:
        if label in label_votes:
            label_votes[label] += 1
        else:
            label_votes[label] = 1
    
    return max(label_votes, key=label_votes.get)

In [14]:
# Muat data latih
train_data, train_labels = load_data_from_json_and_images(image_train_folder, json_train_file)

# Muat data uji
test_data, test_labels = load_data_from_json_and_images(image_test_folder, json_test_file)


In [15]:
# Evaluasi KNN
k = 5  # Tetangga terdekat
correct_predictions = 0

for i, test_sample in enumerate(test_data):
    predicted_label = knn_classify(test_sample, train_data, train_labels, k)
    print(f"Test Image {i+1}: Predicted Label = {predicted_label}, True Label = {test_labels[i]}")
    
    # Hitung jumlah prediksi benar
    if predicted_label == test_labels[i]:
        correct_predictions += 1

# Akurasi
accuracy = correct_predictions / len(test_labels) * 100
print(f"Accuracy: {accuracy:.2f}%")

Test Image 1: Predicted Label = Blue, True Label = Red
Test Image 2: Predicted Label = Red, True Label = Red
Test Image 3: Predicted Label = Red, True Label = Red
Test Image 4: Predicted Label = Red, True Label = Green
Test Image 5: Predicted Label = Red, True Label = Blue
Test Image 6: Predicted Label = Red, True Label = Green
Test Image 7: Predicted Label = Red, True Label = Red
Test Image 8: Predicted Label = Red, True Label = Red
Test Image 9: Predicted Label = Blue, True Label = White
Test Image 10: Predicted Label = Green, True Label = Green
Test Image 11: Predicted Label = Red, True Label = Blue
Test Image 12: Predicted Label = Blue, True Label = Green
Test Image 13: Predicted Label = Red, True Label = Green
Test Image 14: Predicted Label = Red, True Label = Red
Test Image 15: Predicted Label = Red, True Label = Blue
Test Image 16: Predicted Label = Red, True Label = Green
Test Image 17: Predicted Label = Black, True Label = Blue
Test Image 18: Predicted Label = Red, True Label 

In [5]:
import os
import json
from skimage import io
from skimage.transform import resize

# Folder yang berisi gambar dan file JSON yang berisi label
image_train_folder = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/train'
image_test_folder = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/valid' 
json_train_file = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/train/annotations_color_valid_accurate.json'  
json_test_file = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/valid/annotations_color_valid_accurate.json'  

# Fungsi untuk memuat data gambar dan label dari file JSON
def load_data_from_json_and_images(image_folder, json_file):
    images = []
    labels = []
    filenames = []  # Untuk menyimpan nama file gambar
    
    # Membaca file JSON yang berisi anotasi
    with open(json_file) as f:
        data = json.load(f)

    # Membaca gambar dan label dari JSON
    for annotation in data['annotations']:
        image_filename = data['images'][annotation['image_id']]['file_name']  # Ambil nama file gambar berdasarkan image_id
        label = annotation['color_label']  # Label yang berhubungan dengan gambar
        
        # Memuat gambar dari folder
        image_path = os.path.join(image_folder, image_filename)
        image = io.imread(image_path)
        
        # Ubah ukuran gambar jika perlu (agar ukuran konsisten)
        image_resized = resize(image, (64, 64))  # Mengubah ukuran gambar menjadi 64x64
        
        # Flatten gambar menjadi array 1D
        image_flattened = image_resized.flatten()
        
        images.append(image_flattened)
        labels.append(label)
        filenames.append(image_filename)
    
    return images, labels,filenames

# Fungsi untuk menghitung jarak Euclidean antara dua vektor
def euclidean_distance(vec1, vec2):
    return sum((a - b) ** 2 for a, b in zip(vec1, vec2)) ** 0.5

# Fungsi KNN
def knn_classify(test_data, train_data, train_labels, k=3):
    distances = []
    
    # Hitung jarak antara data uji dan semua data latih
    for i in range(len(train_data)):
        dist = euclidean_distance(test_data, train_data[i])
        distances.append((dist, train_labels[i]))
    
    # Urutkan jarak dan ambil k terdekat
    distances.sort(key=lambda x: x[0])
    k_nearest = distances[:k]
    
    # Voting label terbanyak
    label_votes = {}
    for _, label in k_nearest:
        if label in label_votes:
            label_votes[label] += 1
        else:
            label_votes[label] = 1
    
    # Kembalikan label dengan jumlah suara terbanyak
    return max(label_votes, key=label_votes.get)

# Muat data latih
train_data, train_labels = load_data_from_json_and_images(image_train_folder, json_train_file)

# Muat data uji
test_data, test_labels = load_data_from_json_and_images(image_test_folder, json_test_file)

# Evaluasi KNN
k = 5  # Tetangga terdekat
correct_predictions = 0

for i, test_sample in enumerate(test_data):
    predicted_label = knn_classify(test_sample, train_data, train_labels, k)
    
    # Cetak nama file gambar, label prediksi, dan label yang benar
    print(f"Test Image {test_filenames[i]}: Predicted Label = {predicted_label}, True Label = {test_labels[i]}")
    
    # Hitung jumlah prediksi benar
    if predicted_label == test_labels[i]:
        correct_predictions += 1

# Akurasi
accuracy = correct_predictions / len(test_labels) * 100
print(f"Accuracy: {accuracy:.2f}%")


ValueError: too many values to unpack (expected 2)

In [6]:
import os
import json
import numpy as np
from skimage import io
from skimage.transform import resize

# Folder yang berisi gambar dan file JSON yang berisi label
image_train_folder = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/train'
image_test_folder = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/valid' 
json_train_file = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/train/annotations_color_valid_accurate.json'  
json_test_file = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/valid/annotations_color_valid_accurate.json'  

# Fungsi untuk memuat data gambar dan label dari file JSON
def load_data_from_json_and_images(image_folder, json_file):
    images = []
    labels = []
    filenames = []  # Untuk menyimpan nama file gambar
    
    # Membaca file JSON yang berisi anotasi
    with open(json_file) as f:
        data = json.load(f)

    # Membaca gambar dan label dari JSON
    for annotation in data['annotations']:
        image_filename = data['images'][annotation['image_id']]['file_name']  # Ambil nama file gambar berdasarkan image_id
        label = annotation['color_label']  # Label yang berhubungan dengan gambar
        
        # Memuat gambar dari folder
        image_path = os.path.join(image_folder, image_filename)
        image = io.imread(image_path)
        
        # Ubah ukuran gambar jika perlu (agar ukuran konsisten)
        image_resized = resize(image, (64, 64))  # Mengubah ukuran gambar menjadi 64x64
        
        # Flatten gambar menjadi array 1D
        image_flattened = image_resized.flatten()
        
        images.append(image_flattened)
        labels.append(label)
        filenames.append(image_filename)  # Simpan nama file gambar
    
    return np.array(images), labels, filenames  # Kembalikan filenames juga

# Fungsi Euclidean Distance
def euclidean_distance(vec1, vec2):
    return np.sqrt(np.sum((vec1 - vec2) ** 2))

# Fungsi KNN Manual
def knn_classify(test_data, train_data, train_labels, k=3):
    distances = []
    for i in range(len(train_data)):
        dist = euclidean_distance(test_data, train_data[i])
        distances.append((dist, train_labels[i]))
    
    # Urutkan berdasarkan jarak dan ambil k terdekat
    distances.sort(key=lambda x: x[0])
    k_nearest = distances[:k]
    
    # Voting label terbanyak
    label_votes = {}
    for _, label in k_nearest:
        if label in label_votes:
            label_votes[label] += 1
        else:
            label_votes[label] = 1
    
    return max(label_votes, key=label_votes.get)

# Muat data latih
train_data, train_labels, train_filenames = load_data_from_json_and_images(image_train_folder, json_train_file)

# Muat data uji
test_data, test_labels, test_filenames = load_data_from_json_and_images(image_test_folder, json_test_file)

# Evaluasi KNN
k = 5  # Tetangga terdekat
correct_predictions = 0

for i, test_sample in enumerate(test_data):
    predicted_label = knn_classify(test_sample, train_data, train_labels, k)
    
    # Cetak nama file gambar, label prediksi, dan label yang benar
    print(f"Test Image {test_filenames[i]}: Predicted Label = {predicted_label}, True Label = {test_labels[i]}")
    
    # Hitung jumlah prediksi benar
    if predicted_label == test_labels[i]:
        correct_predictions += 1

# Akurasi
accuracy = correct_predictions / len(test_labels) * 100
print(f"Accuracy: {accuracy:.2f}%")


Test Image 1-123204001-OCR-LF-C01_jpg.rf.3ebf94d4d30d7766258de443d5b1dc32.jpg: Predicted Label = Other, True Label = Red
Test Image 1-123303001-OCR-LB-C02_jpg.rf.e20957316647a9514bc278222c0724a9.jpg: Predicted Label = Red, True Label = Red
Test Image 1-123337001-OCR-AH-A01_jpg.rf.9b3ee4fbba917ec73542a59e6c56097b.jpg: Predicted Label = Other, True Label = Other
Test Image 1-123402001-OCR-LB-C02_jpg.rf.f381c410f3cf6e1e03a1d9887902bbad.jpg: Predicted Label = Red, True Label = Blue
Test Image 1-123402001-OCR-RF-D01_jpg.rf.bb107fbd3c8e40bef2101f8e0f6cb9fa.jpg: Predicted Label = Blue, True Label = Blue
Test Image 1-123407001-OCR-AS-B01_jpg.rf.a530131fafebf7e7ef20c0479bae3fa2.jpg: Predicted Label = Black, True Label = Other
Test Image 1-123407001-OCR-LF-C01_jpg.rf.f5582352ba9b9f0972b1ca42a39cec28.jpg: Predicted Label = Other, True Label = Other
Test Image 1-123504001-OCR-LB-C02_jpg.rf.25a9baf656457e9fda992b3e2d90e70d.jpg: Predicted Label = Red, True Label = Red
Test Image 1-123608001-OCR-LB-C